# General lib imports

In [2]:
import pandas as pd
import numpy as np

# Model class

In [18]:
class handmade_nn ():
    '''
    hand-made version of neural network
    so far, the possibilities are :
    
        - layers activation functions : 'linear', 'relu', 'sigmoid', 'tanh', 'softmax'
    
        - loss functions : 'mse', 'mae', 'binary_crossentropy', 'categorical_crossentropy'
    
        - solver : SGD without momentum
    '''
    def __init__ (self, input_dim=0, loss=None, learning_rate=0.01):
        self.weights=[]
        self.bias=[]
        self.activation_types=[]
        self.input_dim=input_dim
        self.loss=loss
        self.learning_rate=learning_rate
    
    def compute_activation(self,X,activation_type):
        # Defining activation functions
        # Takes a nparray or a single value
        # Returns in the same format
        if activation_type == 'relu':
            return np.maximum(x,0)
        elif activation_type == 'sigmoid':
            return 1/(1+np.exp(-x))
        elif activation_type == 'tanh':
            return np.tanh(x)
        elif activation_type == 'linear':
            return x
        elif activation_type == 'softmax':
            exp_x = np.exp(x)
            return exp_x / exp_x.sum()
        
        #raise error if unknown type
        else:
            raise ValueError(f'Unknow activation type {activation_type}. Supported types : linear, relu, sigmoid, tanh, softmax')
            
    def set_input_dim (self,input_dim):
        self.input_dim = input_dim
        
    def set_loss (self,loss):
        self.loss = loss
        
    def set_learning_rate (self,learning_rate):
        self.learning_rate = learning_rate
        
    def add_dense_layer (self, n_neurons, activation_type):
        #check if the input_dim is set
        if self.input_dim == 0:
            raise ValueError('input_dim = 0 . Use set_input_dim before creating first layer')
            
        #get the size of the input os this layer
        if len(self.bias) == 0:
            previous_dim=self.input_dim
        else:
            previous_dim=(self.bias[-1].shape[0])
            
        #initialize the layer parameters 
        self.weights.append(np.zeros((n_neurons, previous_dim)))
        self.bias.append(np.expand_dims(np.zeros(n_neurons), axis=0))
        self.activation_types.append(activation_type)
        
        #test the activation type
        self.activation(0, activation_type)
        
    def predict (self,X):
        #converting DataFrames, lists or lists of lists to nparray
        X = np.array(X)
        
        #deal with 1D inputs to forge a 1 * n_features 2D-array
        if len(X.shape) == 1:
            X = np.expand_dims(X, axis = 0)
            
        #raise errors for unconsistant inputs
        if len(X.shape) > 2:
            raise ValueError('X vector dimension too high. Must be 2 max')
        if X.shape[1] != self.input_dim:
            raise ValueError(f'Unconsistent number of features. The network input_dim is {self.input_dim}')
            
        #compute the prediction
        for layer_index, activation_type in enumerate(self.activation_types):
            activation_input=np.dot(self.weights[layer_index],X.T) + self.bias[layer_index].T
            X = self.compute_activation(activation_input, activation_type).T
        return X
    
    def compute_metric (self, y, y_pred, metric):
        # Defining loss and metric functions
        # Takes a nparray, a list or a single value
        # Always returns a scalar : in case of multioutputs (y.shape[1]>1), 
        #     uniform average of the errors along axis1
        
        #converting DataFrames, lists or lists of lists to nparray
        y = np.array(y)
        y_pred = np.array(y_pred)
        
        #deal with 1D inputs to forge a n_samples * 1 2D-array
        if len(y.shape) == 1:
            y = np.expand_dims(y, axis = 1)
        if len(y_pred.shape) == 1:
            y_pred = np.expand_dims(y_pred, axis = 1)
            
        #raise errors for unconsistant inputs
        if len(y.shape) > 2:
            raise ValueError('y vector dimension too high. Must be 2 max')
        if len(y_pred.shape) > 2:
            raise ValueError('y_pred vector dimension too high. Must be 2 max')
        if y.shape != y_pred.shape:
            raise ValueError(f'unconsistent vectors dimensions during scoring : y.shape= {y.shape} and y_pred.shape= {y_pred.shape}')
        
        #compute loss funtions
        if metric == 'mse':
            return np.square(y-y_pred).mean()
        if metric == 'mae':
            return np.abs(y-y_pred).mean()
        if metric == 'binary_crossentropy'
            if y.shape[1]>1:
                raise ValueError('y vector dimension too high. Must be 1 max for binary_crossentropy')
            y_pred = y_pred[:,-1]
            return -(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).mean()
        if metric == 'categorical_crossentropy':
            pass
        
        # compute other metrics functions
            

# Tests

## add_dense_layer method tests

### raise ValueError if no input_dim

In [3]:
from unittest import TestCase
my_first_nn=handmade_nn()

test=TestCase()
with test.assertRaises(ValueError) as context:
    my_first_nn.add_dense_layer(5,'relu')
assert 'input_dim = 0 . Use set_input_dim before creating first layer' in str(context.exception),\
    "no or wrong Exception raised when adding first layer to a network without setting input_dim"

### raise ValueError for unvalid activation type

In [4]:
from unittest import TestCase
my_first_nn=handmade_nn(5)

test=TestCase()
with test.assertRaises(ValueError) as context:
    my_first_nn.add_dense_layer(10,'typo_error')
assert 'Unknow activation type' in str(context.exception),\
    "no or wrong Exception raised when inputing an unvalid activation_type"    

## predict method tests

### X format acceptance

#### handling with a list as an input

In [5]:
my_first_nn=handmade_nn(5)
# Empty neural network : just a pass-through for 5-values inputs
assert my_first_nn.predict([2,3,2,3,4]).shape == (1,5),\
    "list not supported as an input for predict"

#### handling with a list of lists as input

In [6]:
my_first_nn=handmade_nn(5)
# Empty neural network : just a pass-through for 5-values inputs
assert my_first_nn.predict([[2,3,2,3,4],[-2,-1,1,3,4]]).shape == (2,5),\
    "list of list not supported as an input for predict"

#### handling with a 1D-array

In [7]:
my_first_nn=handmade_nn(5)
my_first_nn.add_dense_layer(10, 'linear')
my_first_nn.weights[0] = np.vstack((np.identity(5),np.zeros((5,5))))
assert my_first_nn.predict(np.array([-2,-1,2,3,4])).shape == (1,10),\
    "1-D array not supported as an input for predict"

#### handling with a 2D-array (most common case)

In [8]:
my_first_nn=handmade_nn(5)
my_first_nn.add_dense_layer(10, 'linear')
my_first_nn.weights[0] = np.vstack((np.identity(5),np.zeros((5,5))))
assert my_first_nn.predict(np.array([[-2,-1,2,3,4],[-12,-11,12,13,14]])).shape == (2,10),\
    f"the shape of the prediction for a 2*5 X input by a network having 10neurons on last layer should be 2*10"

#### raise error for 3D-array or more

In [9]:
from unittest import TestCase
my_first_nn=handmade_nn(5)
# Empty neural network : just a pass-through for 5-values inputs

test=TestCase()
with test.assertRaises(ValueError) as context:
    my_first_nn.predict(np.array([[[1,1],[1,2],[1,3],[1,4],[1,5]],
                                 [[2,1],[2,2],[2,3],[3,4],[3,5]]]))
assert 'X vector dimension too high' in str(context.exception),\
    "no or wrong Exception raised when inputing a 3D-array in predict method"    

#### raise error for unconsitant X vs. input_dim

In [10]:
from unittest import TestCase
my_first_nn=handmade_nn(5)
# Empty neural network : just a pass-through for 5-values inputs

test=TestCase()
with test.assertRaises(ValueError) as context:
    my_first_nn.predict(np.array([[1,1],[1,2],[1,3],[1,4],[1,5]]))
assert 'Unconsistent number of features' in str(context.exception),\
    "no or wrong Exception raised when inputing a X with unconsistant size vs. network input_dim"    

### checking behaviour of each activation function

#### relu

In [11]:
my_first_nn=handmade_nn(5)
my_first_nn.add_dense_layer(10, 'relu')
my_first_nn.weights[0] = np.vstack((np.identity(5),np.zeros((5,5))))
assert (my_first_nn.predict([-2,-1,2,3,4]) ==\
        np.array([[0., 0., 2., 3., 4., 0., 0., 0., 0., 0.]]))\
        .all(), "uncorrect relu function behaviour"

#### linear 

In [12]:
my_first_nn=handmade_nn(5)
my_first_nn.add_dense_layer(10, 'linear')
my_first_nn.weights[0] = np.vstack((np.identity(5),np.zeros((5,5))))
assert (my_first_nn.predict([-2,-1,2,3,4]) ==\
        np.array([[-2., -1.,  2.,  3.,  4.,  0.,  0.,  0.,  0.,  0.]]))\
        .all(), "uncorrect linear function behaviour"

#### sigmoid

In [13]:
my_first_nn=handmade_nn(5)
my_first_nn.add_dense_layer(10, 'sigmoid')
my_first_nn.weights[0] = np.vstack((np.identity(5),np.zeros((5,5))))
assert (np.round(my_first_nn.predict([-2,-1,2,3,4]), 8) ==\
        np.array([[0.11920292, 0.26894142, 0.88079708, 0.95257413, 0.98201379,
        0.5       , 0.5       , 0.5       , 0.5       , 0.5       ]]))\
        .all(), "uncorrect sigmoid function behaviour"

#### tanh

In [14]:
my_first_nn=handmade_nn(5)
my_first_nn.add_dense_layer(10, 'tanh')
my_first_nn.weights[0] = np.vstack((np.identity(5),np.zeros((5,5))))
assert (np.round(my_first_nn.predict([-2,-1,2,3,4]), 8) ==\
        np.array([[-0.96402758, -0.76159416,  0.96402758,  0.99505475,  0.9993293 ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]]))\
        .all(), "uncorrect tanh function behaviour"

#### softmax

In [15]:
my_first_nn=handmade_nn(5)
my_first_nn.add_dense_layer(10, 'softmax')
my_first_nn.weights[0] = np.vstack((np.identity(5),np.zeros((5,5))))
assert (np.round(my_first_nn.predict([-2,-1,2,3,4]), 8) ==\
        np.array([[0.00154535, 0.00420069, 0.08437311, 0.2293499 , 0.62343766,
        0.01141866, 0.01141866, 0.01141866, 0.01141866, 0.01141866]]))\
        .all(), "uncorrect softmax function behaviour"

In [8]:
np.expand_dims(np.array(pd.Series([2,4,5])),axis=1).shape

(3, 1)

In [14]:
y_true = np.array([[0.5, 1],[-1, 1],[7, -6]])
y_pred = np.array([[0, 2],[-1, 2],[8, -5]])
mse=0.708

In [18]:
ess=y_true-y_pred

In [20]:
ess

array([[ 0.5, -1. ],
       [ 0. , -1. ],
       [-1. , -1. ]])

In [24]:
np.square(ess).mean()

0.7083333333333334

In [27]:
y=np.array([[1],[0],[1]])
y.shape

(3, 1)

In [28]:
y_pred=np.array([[0.9],[0.2],[0.95]])
y_pred

array([[0.9 ],
       [0.2 ],
       [0.95]])

In [35]:
-(y*np.log(y_pred)+(1-y)*np.log(1-y_pred)).mean()

0.12659912045319552

In [32]:
(1-y_pred)

array([[0.1 ],
       [0.8 ],
       [0.05]])